# Marker annotation by calculating escore of markers

#### Note that we are in Python

#### Working directory must contain subdirectories,supp_data

#### supp_data/ should contain the files, which are available on Github (github.com/Hsu-Che-Wei/COPILOT):

    hvg_integrated.h5 (get it by running through notebook 3-1)  
    curated.genes.txt
    hvg_ids.txt (get it by running through notebook 3-2)
    r_cells.txt
    umap50.txt (get it by running through notebook 3-2)
    umap.txt (get it by running through notebook 3-2)


### 1. Import all needed functions

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from escoring.enrichment_scoring import calculate_escores
from escoring.enrichment_scoring import permute, sig_interval
from escoring.support_funcs import load_sparse_h5, pairwise_similarities
from escoring.support_funcs import sig_dictionary

### 2. Load the data

#### Expression matrix

In [99]:
HVG = load_sparse_h5("counts","./supp_data/hvg_integrated.h5")

In [100]:
type(HVG)

scipy.sparse.csr.csr_matrix

In [101]:
# Escoring accepts matrices where rows are cells and columns are genes. So one will need to transpose the matrix if the matrix is gene-by-cell
HVG = HVG.T

In [102]:
HVG.shape

(110427, 17513)

#### Markers 

In [104]:
fname = "./supp_data/curated.genes.txt"
with open(fname, "r") as f:
    cols_to_keep = [int(cell.strip("\n")) for cell in f.readlines()]
f.close()

In [105]:
HVG = HVG[:, cols_to_keep]

In [106]:
HVG.shape

(110427, 44)

In [107]:
fname = "./supp_data/hvg_ids.txt"
with open(fname, "r") as f:
    hvg_names = [gene.strip("\n") for gene in f.readlines()]
f.close()

In [110]:
hvg_names = list( hvg_names[i] for i in cols_to_keep )

In [111]:
len(hvg_names)

44

#### Reference cells

In [112]:
# It is just the index of cells, in this case, all the cells are considered
fname = "./supp_data/r_cells.txt" 
with open(fname, "r") as f:
    r_cells = [int(cell.strip("\n")) for cell in f.readlines()]
f.close()

In [113]:
r_cells = np.arange(HVG.shape[0])

In [114]:
r_cells = r_cells.tolist()

In [116]:
len(r_cells)

110427

#### 50 UMAP dimensions for similarity calculations

In [117]:
UMAP_50 = np.loadtxt("./supp_data/umap50.txt")  # or load your preferred representation

#### 2D UMAP for visualization

In [118]:
UMAP = np.loadtxt("./supp_data/umap.txt")

### 3. Enrichment scoring

In [2]:
# Use RBF-kernel
metric = "rbf"  
gamma = 0.8  # only use if laplacian, sigmoid or rbf and replace by wished value
S = pairwise_similarities(UMAP_50, r_cells, metric=metric,
                          metric_params={"gamma": gamma}  # only use if needed
                         )

In [120]:
S.shape

(110427, 110427)

In [121]:
# Calculate the enrichment scores
escores = calculate_escores(HVG, r_cells, S=S, optim_over="rows", scale_exp=False)

Start enrichment scoring using 32 CPUs
Creating process pool
Run enrichment scoring
Enrichment scoring complete


The escores dataframe is a dataframe of size genes x r_cells. The order of genes is preserved, so you can map them back to the indices of the genes in the original data. Take care here that in Python, counting starts at 0 and not 1. If you need any help here, let me know. Below, I manually set the gene names to the index.

In [122]:
escores.index = hvg_names

In [123]:
escores.index

Index(['AT4G00490', 'AT1G62510', 'AT5G07990', 'AT5G55250', 'AT1G61590',
       'AT4G02090', 'AT3G11550', 'AT2G27370', 'AT5G06200', 'AT5G57620',
       'AT5G42180', 'AT1G33090', 'AT3G09330', 'AT5G49270', 'AT1G48930',
       'AT1G69240', 'AT1G45545', 'AT4G37160', 'AT1G79840', 'AT2G37260',
       'AT1G65310', 'AT1G79430', 'AT5G62940', 'AT2G37090', 'AT5G12870',
       'AT3G09070', 'AT5G43810', 'AT5G26930', 'AT3G26120', 'AT3G43270',
       'AT1G68470', 'AT1G55440', 'AT1G11330', 'AT1G26790', 'AT3G05180',
       'AT3G55930', 'AT5G36880', 'AT1G23210', 'AT5G57640', 'AT3G60650',
       'AT1G13620', 'AT1G17400', 'AT1G55200', 'AT5G45210'],
      dtype='object')

In [124]:
# Permute the dataframe. This takes a little while.
n = 100  # how many times to permute the dataframe
seed = 42  # set this for reproducibility
P = permute(HVG, n=n, seed=seed, axis=0)

Calculate scores for the permuted expression values. Make sure to pass the permuted dataframe and keep all other parameters the same.

In [125]:
pscores = calculate_escores(P, r_cells, S=S, optim_over="rows", scale_exp=False)

Start enrichment scoring using 32 CPUs
Creating process pool
Run enrichment scoring
Enrichment scoring complete


In [127]:
# Determine the significance cut-offs
n_sds = 5  # the number of SDs away from the mean for significance
cutoffs = sig_interval(pscores, n_sds=n_sds)

In [128]:
# Get a dictionary of significant genes per cell
sigs = sig_dictionary(escores, cutoffs, retrieve="cols")

In [129]:
# Save the output
df = pd.DataFrame({key: pd.Series(value) for key, value in sigs.items()})
df.to_csv("./supp_data/escoring.curated.marker.anno.nsds5.csv", encoding='utf-8', index=False)

In [130]:
# Save the output
escores.to_csv("./supp_data/escoring.curated.marker.csv", encoding='utf-8', index=False)